In [1]:
!pip install langchain google-search-results
!pip install duckduckgo-search
!pip install numexpr
!pip install youtube_search
!pip install chardet
#!pip uninstall charset-normalizer -y
!pip install charset-normalizer
#!pip uninstall typing-extensions -y
!pip install typing-extensions
!pip install --upgrade typing_extensions

In [2]:
from langchain import HuggingFacePipeline
from langchain.agents import load_tools, initialize_agent, Tool
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun, YouTubeSearchTool
from langchain.utilities import WikipediaAPIWrapper
from huggingface_hub import login
import gradio as gr
import torch
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
import os

import api_tokens

/home/dlopes/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`AnnotionFormat` is deprecated and will be removed in v4.38. Please use `transformers.image_utils.AnnotationFormat` instead.
2023-12-15 16:50:28.932487: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-15 16:50:28.966667: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 16:50:28.966696: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT

In [3]:
os.environ['HF_DATASETS_CACHE'] = api_tokens.HF_DATASETS_CACHE

login(token=api_tokens.HUGGINGFACEHUB_API_TOKEN)

embeddings_name = 'sentence-transformers/all-MiniLM-L6-v2'
model_name = 'meta-llama/Llama-2-7b-chat-hf'

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.


Token is valid (permission: read).
Your token has been saved to /home/dlopes/.cache/huggingface/token
Login successful


In [4]:
def build_langchain_pipeline(model_name: str) -> HuggingFacePipeline:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name,
                                                device_map='auto',
                                                torch_dtype=torch.float16,
                                                use_auth_token=True,
                                                load_in_8bit=True,
                                                )

    pipe = pipeline("text-generation",
                    model=model,
                    tokenizer= tokenizer,
                    torch_dtype=torch.bfloat16,
                    device_map="auto",
                    max_new_tokens = 4096, # maxiumum number of tokens to generate allowed for the model Llama-2-7b-chat-hf
                    do_sample=True,
                    top_k=10,
                    num_return_sequences=1,
                    eos_token_id=tokenizer.eos_token_id
                    )

    llm = HuggingFacePipeline(pipeline=pipe,
                                model_kwargs={
                                    'temperature':0
                                },
                            )
    
    return llm

### Langchain Tools
* https://www.datacamp.com/tutorial/building-a-gpt-model-with-browsing-capabilities-using-lang-chain-tools 
* Addresses the lack of support for ChatGPT to use its browsing capabilities when being interacted with via the API.
* Langchain Tools allows us to create our own ChatGPT API with browsing capabilities.
* In general, Langchain Tools provide a prompting technique called **ReAct Prompting**, which divides the model's processing into two stages: **Reasoning stage** and **Acting stage**.
* In the Reasoning stage, the model takes time to consider the best strategy for arriving at a response.
* In the Acting stage, the model executes the plan formulated in the previous stage to obtain the actual response.
* This way, Langchain Tools enable the model to consult external sources when responding to a user's prompt.
* **Agents:** https://python.langchain.com/docs/modules/agents/agent_types/
* **Examples of Langchain Tools:**
    * Google Search, Drive and Scholar
    * ArXiV database
    * shell extension
    * native ChatGPT plugins
    * Full list here: https://python.langchain.com/docs/integrations/tools 


In [5]:
llm = build_langchain_pipeline(model_name)

/home/dlopes/anaconda3/envs/pytorch2/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.43s/it]


#### Google Search

* Get API key for Google Serper Tool (SerpApi) here: https://serpapi.com/users/welcome
* Has only 100 free calls per month

In [6]:
os.environ["SERPAPI_API_KEY"] = api_tokens.SERP_API_TOKEN

In [7]:
tool_names =['serpapi','llm-math']
tools=load_tools(tool_names, llm)

The prime minister of India, Narendra Modi, is 73 years old.
The president of America, Joe Biden, is 81 years old.

# TODO: This means that we are obtaining the wrong answer, and its also not giving me the square root of the age.

In [8]:
agent = initialize_agent(tools,llm,agent='zero-shot-react-description', verbose=True)
agent.run("who is older? the president of America or PM of India.Take the older man's age and give me its square root with an action input of sqrt.")



> Entering new AgentExecutor chain...
 Let me check their ages and compare them.
Action: Search (age of President of America and PM of India)
Action Input: age of President of America and PM of India
Observation: Search (age of President of America and PM of India) is not a valid tool, try one of [Search, Calculator].
Thought:

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


 Okay, let me use the calculator to find the square root of their ages.
Action: Calculator (age of President of America and PM of India)
Action Input: 78 and 62
Observation: Calculator (age of President of America and PM of India) is not a valid tool, try one of [Search, Calculator].
Thought: Oh, my mistake. Let me try again.
Action: Search (age of President of America and PM of India)
Action Input: age of President of America and PM of India
Observation: Search (age of President of America and PM of India) is not a valid tool, try one of [Search, Calculator].
Thought: Okay, I'll try the calculator again.
Action: Calculator (age of President of America and PM of India)
Action Input: 78 and 62
Observation: Calculator (age of President of America and PM of India) is not a valid tool, try one of [Search, Calculator].
Thought: Hmm, this is strange. Let me check the ages again.
Action: Search (age of President of America and PM of India)
Action Input: age of President of America and PM of I

'8.6\n\nThe square root of 78 and 62 is 8.6.'

#### DuckDuckGo

In [9]:
ddg_search = DuckDuckGoSearchRun()

In [10]:
ddg_search.run("What are langchain tools?")

"description: a short instruction manual that explains when and why the agent should use the tool. args_schema: Communicates the interface of the tool for the agent. It typically draws from the wrapped function's signature and permits additional validation logic on tool inputs. _run and _arun functions: These define the tool's inner workings. What is Langchain? LangChain is a framework for developing applications powered by language models. Their framework enables you to build layered LLM-powered applications that are context-aware and able to interact dynamically with their environment as agents, leading to simplified code for you and a more dynamic user experience for your customers. At its core, LangChain is an innovative framework tailored for crafting applications that leverage the capabilities of language models. It's a toolkit designed for developers to create applications that are context-aware and capable of sophisticated reasoning. LangChain is a versatile Python library that

In [11]:
tools = [
   Tool(
       name="DuckDuckGo Search",
       func=ddg_search.run,
       description="Useful to browse information from the Internet.",
   )
]

#### Wikipedia

In [12]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

ImportError: Could not import wikipedia python package. Please install it with `pip install wikipedia`.

In [ ]:
tools.append(
   Tool(
       name="Wikipedia Search",
       func=wikipedia.run,
       description="Useful when users request biographies or historical moments.",
   )
)

#### Youtube

In [ ]:
youtube = YouTubeSearchTool()

In [ ]:
tools.append(
   Tool(
       name="Youtube Search",
       func=youtube.run,
       description="Useful for when the user explicitly asks you to look on Youtube.",
   )
)

#### Browsing Capabilities

In [ ]:
agent = initialize_agent(
   tools, llm, agent="zero-shot-react-description", verbose=True
)

In [ ]:
agent.run(
   "Harry Styles biography"
)

In [ ]:
agent.run(
   "Harry Styles biography"
)

In [ ]:
agent = initialize_agent(
   tools, llm, agent="zero-shot-react-description"
)

agent.run(
   "Can you point me to the Procrastination TED talk by Tim Urban?"
)